# Topic Modelling with Gensim (Python)

One of the primary applications of natural language processing is to automatically extract what topics people are discussing from large volumes of text. Some examples of large text could be feeds from social media, customer reviews of hotels, movies, etc, user feedbacks, news stories, e-mails of customer complaints etc.

Knowing what people are talking about and understanding their problems and opinions is highly valuable to businesses, administrators, political campaigns. And it’s really hard to manually read through such large volumes and compile the topics.
Thus is required an automated algorithm that can read through the text documents and automatically output the topics discussed.

In this project, I will take a real example of the ’20 Newsgroups’ dataset and use LDA to extract the naturally discussed topics.

## Import packages

The core packages I'm going to use in this project are re, gensim, spacy and pyLDAvis. On top of that I'll use pandas, numpy and matplotlib.

I will need the stopwords from NLTK and spacy's en_core_web_sm model for text pre-processing. Later in the project spacy will be used for lemmatization. 

In [38]:
import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

ModuleNotFoundError: No module named 'gensim.models.wrappers'

## LDA approach

LDA’s approach to topic modeling is that it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.

Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

A topic is a collection of dominant keywords that are typical representatives. Just by looking at the keywords, you can identify what the topic is all about.

The following are key factors to obtaining good segregation topics:

1. The quality of text processing.
2. The variety of topics the text talks about.
3. The choice of topic modeling algorithm.
4. The number of topics fed to the algorithm.
5. The algorithms tuning parameters.

## Prepare Stopwords

In [13]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Import Newsgroups Data

I will be working with the 20-Newsgroups dataset for this project. This version of dataset contains about 11k posts from 20 different topics. This is available at newsgroups.json.

In [14]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## Remove emails and newline characters

In [15]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


After removing the emails and extra spaces, the text is still not ready for the LDA to work with. I need to break down each sentence into a list of words through tokenization, while clearing up all the messy text in the process.

I'm going to use Gensim’s simple_preprocess for this.

## Tokenize words and Clean-up text

I'm going to tokenize each sentence into a list of words, removing punctuations and unnecessary characters.

Using Gensim's simple_preprocess(), I'm going to set deacc=True to remove the punctuations.

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are three words frequently occurring.
Some examples in current dataset are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


Now, I'm going to create functions to remove stopwords, make bigrams and lemmatization.

Lemmatization is a process of converting a word to its root word. For example: the lemma of the word 'machines' is 'machine', 'walking' is 'walk' and so on.

In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# In terminal type: python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## Create the Dictionary and Corpus needed for Topic Modeling

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

Let's see what word a given id corresponds to, pass the id as a key to the dictionary.

In [28]:
id2word[0]

'addition'

In [29]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('wonder', 1),
  ('year', 1)]]

## Building the topic 

Now I have everything to run the LDA model. In addition to the corpus and dictionary, I need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [30]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## View the topics in LDA model

The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

Let's see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics().

In [31]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.266*"drive" + 0.066*"disk" + 0.057*"tape" + 0.044*"originator" + '
  '0.033*"external" + 0.021*"bio" + 0.020*"compile" + 0.016*"super" + '
  '0.016*"plug" + 0.014*"receiver"'),
 (1,
  '0.040*"kill" + 0.029*"attack" + 0.029*"war" + 0.026*"soldier" + '
  '0.023*"village" + 0.021*"armenian" + 0.019*"death" + 0.018*"murder" + '
  '0.016*"turkish" + 0.015*"people"'),
 (2,
  '0.041*"map" + 0.035*"canadian" + 0.029*"encourage" + 0.024*"reflect" + '
  '0.023*"enable" + 0.022*"partner" + 0.022*"entity" + 0.021*"pat" + '
  '0.019*"au" + 0.018*"translate"'),
 (3,
  '0.031*"report" + 0.027*"week" + 0.025*"player" + 0.021*"power" + '
  '0.016*"israeli" + 0.016*"national" + 0.016*"wire" + 0.015*"member" + '
  '0.014*"center" + 0.013*"publish"'),
 (4,
  '0.060*"fan" + 0.040*"clearly" + 0.031*"hot" + 0.030*"night" + 0.028*"mine" '
  '+ 0.023*"cap" + 0.022*"motto" + 0.022*"german" + 0.022*"water" + '
  '0.021*"recall"'),
 (5,
  '0.096*"drug" + 0.056*"recommend" + 0.056*"printer" + 0.051*"gate

## Interpretation

Topic 0 is a represented as '0.266*"drive" + 0.066*"disk" + 0.057*"tape" + 0.044*"originator" + 0.033*"external" + 0.021*"bio" + 0.020*"compile" + 0.016*"super" + 0.016*"plug" + 0.014*"receiver"'

It means the top 10 keywords that contribute to this topic are: ‘drive’, ‘disk’, ‘tape’.. and so on and the weight of ‘drive’ on topic 0 is 0.266.

The weights reflect how important a keyword is to that topic.
Looking at these keywords, I'd summarize first topic to, maybe, 'biotechnology'.

## Perplexity and topic coherence
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. A low perplexity indicates the probability distribution is good at predicting the sample.

Topic coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. 

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.519608360116582

Coherence Score:  0.5206979792104978


## Visualize the topics-keywords

Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. I'm going to use pyLDAvis package’s interactive chart because it is designed to work well with jupyter notebooks.

In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.356828 -0.049142       1        1  27.127117
15     0.333085  0.081698       2        1  25.547115
17     0.294673  0.077711       3        1  14.993518
6      0.158506  0.238777       4        1   5.202736
11    -0.027635 -0.001088       5        1   4.526994
8      0.131776 -0.228452       6        1   3.717997
18     0.095179  0.142879       7        1   3.697516
3      0.065768 -0.091741       8        1   3.620640
10     0.038768 -0.228633       9        1   2.693555
1      0.022535 -0.206196      10        1   1.998243
13    -0.038795  0.063606      11        1   1.423677
14    -0.077514  0.004613      12        1   1.250661
12    -0.117882  0.036705      13        1   1.054659
4     -0.150303  0.017897      14        1   0.796776
0     -0.143825  0.019712      15        1   0.725553
19    -0.155538  0.052743      16        1   0.524508
5     -0.185682  0.017443      17        1   0.360110
9     -0.194654  0.017535      18        1   0.325958
2     -0.195231  0.017252      19        1   0.305915
16    -0.210060  0.016680      20        1   0.106753, topic_info=              Term          Freq         Total Category  logprob  loglift
4434            ax  39423.000000  39423.000000  Default   30.000  30.0000
18            line  17893.000000  17893.000000  Default   29.000  29.0000
143   organization  10877.000000  10877.000000  Default   28.000  28.0000
380          drive   2716.000000   2716.000000  Default   27.000  27.0000
144         people   8106.000000   8106.000000  Default   26.000  26.0000
...            ...           ...           ...      ...      ...      ...
6383           aid      0.021173      1.260928  Topic20  -11.093   2.7555
6504        winner      0.021173      1.260928  Topic20  -11.093   2.7555
6095         print      0.021173      1.260973  Topic20  -11.093   2.7555
3763        accord      0.021173      1.260984  Topic20  -11.093   2.7555
5620         judge      0.021173      1.260963  Topic20  -11.093   2.7555

[866 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1390      1  0.998699        accept
86        3  0.327064        access
86        4  0.511594        access
86       11  0.159078        access
3180     11  0.982454  access_digex
...     ...       ...           ...
40        2  0.825176          year
40        3  0.005292          year
40        8  0.115207          year
40       10  0.012620          year
40       12  0.041523          year

[1098 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 16, 18, 7, 12, 9, 19, 4, 11, 2, 14, 15, 13, 5, 1, 20, 6, 10, 3, 17])

## Interpretation

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model would have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, would typically have many overlaps, small sized bubbles clustered in one region of the chart.

If I move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

With this model building the project is done. This was my first try and topic assessment. I would like to try using different models in the future and compare their effectiveness.